### Тема: Интеграция REST API с базой данных (PostgreSQL) на Go

1. **Цель урока**:
   - Научиться подключаться к базе данных PostgreSQL из Go.
   - Реализовать взаимодействие API с базой данных для выполнения CRUD-операций.
   - Рассмотреть использование библиотеки GORM для упрощения работы с базой данных.

2. **План урока**:
   1. **Введение в базу данных PostgreSQL**:
      - Объяснение, что такое реляционные базы данных и как работает PostgreSQL.
      - Краткий обзор SQL-запросов (SELECT, INSERT, UPDATE, DELETE).

   2. **Установка и настройка PostgreSQL**:
      - Установка PostgreSQL (если еще не установлен).
      - Создание новой базы данных и таблицы для хранения данных о книгах:
        ```sql
        CREATE DATABASE bookdb;
        \c bookdb;
        CREATE TABLE books (
            id SERIAL PRIMARY KEY,
            title VARCHAR(100),
            author VARCHAR(100)
        );
        ```

   3. **Установка необходимых пакетов Go**:
      - Подключим драйвер для PostgreSQL и библиотеку GORM:
        ```bash
        go get -u gorm.io/gorm
        go get -u gorm.io/driver/postgres
        ```

   4. **Подключение к базе данных и настройка GORM**:
      - Добавим код для соединения с базой данных в `main.go`:

        ```go
        package main

        import (
            "gorm.io/driver/postgres"
            "gorm.io/gorm"
            "log"
        )

        var db *gorm.DB

        func initDB() {
            dsn := "host=localhost user=youruser password=yourpassword dbname=bookdb port=5432 sslmode=disable"
            var err error
            db, err = gorm.Open(postgres.Open(dsn), &gorm.Config{})
            if err != nil {
                log.Fatal("Failed to connect to database:", err)
            }

            // Миграция схемы
            db.AutoMigrate(&Book{})
        }
        ```

   5. **Модели и реализация CRUD-операций с базой данных**:
      - Обновим модель `Book` для использования с GORM и реализуем CRUD-операции:

        ```go
        type Book struct {
            ID     uint   `gorm:"primaryKey" json:"id"`
            Title  string `json:"title"`
            Author string `json:"author"`
        }

        func getBooks(c *gin.Context) {
            var books []Book
            db.Find(&books)
            c.JSON(http.StatusOK, books)
        }

        func getBookByID(c *gin.Context) {
            id := c.Param("id")
            var book Book
            if err := db.First(&book, id).Error; err != nil {
                c.JSON(http.StatusNotFound, gin.H{"message": "book not found"})
                return
            }
            c.JSON(http.StatusOK, book)
        }

        func createBook(c *gin.Context) {
            var newBook Book
            if err := c.BindJSON(&newBook); err != nil {
                c.JSON(http.StatusBadRequest, gin.H{"message": "invalid request"})
                return
            }
            db.Create(&newBook)
            c.JSON(http.StatusCreated, newBook)
        }

        func updateBook(c *gin.Context) {
            id := c.Param("id")
            var updatedBook Book
            if err := c.BindJSON(&updatedBook); err != nil {
                c.JSON(http.StatusBadRequest, gin.H{"message": "invalid request"})
                return
            }
            if err := db.Model(&Book{}).Where("id = ?", id).Updates(updatedBook).Error; err != nil {
                c.JSON(http.StatusNotFound, gin.H{"message": "book not found"})
                return
            }
            c.JSON(http.StatusOK, updatedBook)
        }

        func deleteBook(c *gin.Context) {
            id := c.Param("id")
            if err := db.Delete(&Book{}, id).Error; err != nil {
                c.JSON(http.StatusNotFound, gin.H{"message": "book not found"})
                return
            }
            c.JSON(http.StatusOK, gin.H{"message": "book deleted"})
        }
        ```

   6. **Обновление `main.go` для работы с базой данных**:
      - Добавим вызов функции `initDB()` в `main()` и маршруты для API:

        ```go
        func main() {
            initDB()
            router := gin.Default()

            router.GET("/books", getBooks)
            router.GET("/books/:id", getBookByID)
            router.POST("/books", createBook)
            router.PUT("/books/:id", updateBook)
            router.DELETE("/books/:id", deleteBook)

            router.Run(":8080")
        }
        ```

3. **Практическая часть**:
   - Создать собственную модель данных и интегрировать её с базой данных.
   - Реализовать новые маршруты и операции с другими таблицами.
